In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split


def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 10)
    return files, targets


train_files, train_targets = load_dataset('/kaggle/input/distracteddriverdetection/data/train')

test_files,test_targets= load_dataset('/kaggle/input/distracteddriverdetection/data/test')



names = [item[17:19] for item in sorted(glob("/kaggle/input/distracteddriverdetection/data/train/*/"))]


train_files, valid_files, train_targets, valid_targets = train_test_split(train_files, train_targets, test_size=0.2, random_state=42)


print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d total training categories.' % len(names))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

There are 4662 total images.

There are 2484 training images.
There are 10 total training categories.
There are 622 validation images.
There are 1556 test images.


In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):

    img = image.load_img(img_path, target_size=(224, 224))

    x = image.img_to_array(img)

    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)



In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 


train_tensors = paths_to_tensor(train_files).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255 - 0.5
test_tensors = paths_to_tensor(test_files).astype('float32')/255 - 0.5


100%|██████████| 1556/1556 [00:09<00:00, 161.68it/s]


In [11]:
train_tensors.shape

(2484, 224, 224, 3)

In [12]:
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras import layers
from tensorflow.keras import models
# from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3), kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 128)     32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       131328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 512)       5

In [13]:
model.compile(optimizer= 'adam',loss = 'categorical_crossentropy',metrics= ['accuracy'])

In [15]:
model.fit(train_tensors,train_targets,epochs = 10)

Epoch 1/10
78/78 [==============================] - 294s 4s/step - loss: 2.0500 - accuracy: 0.2621
Epoch 2/10
78/78 [==============================] - 298s 4s/step - loss: 1.1555 - accuracy: 0.5978
Epoch 3/10
78/78 [==============================] - 294s 4s/step - loss: 0.6250 - accuracy: 0.7947
Epoch 4/10
78/78 [==============================] - 302s 4s/step - loss: 0.3806 - accuracy: 0.8752
Epoch 5/10
78/78 [==============================] - 295s 4s/step - loss: 0.2654 - accuracy: 0.9094
Epoch 6/10
78/78 [==============================] - 300s 4s/step - loss: 0.2250 - accuracy: 0.9263
Epoch 7/10
78/78 [==============================] - 295s 4s/step - loss: 0.1736 - accuracy: 0.9436
Epoch 8/10
78/78 [==============================] - 302s 4s/step - loss: 0.1358 - accuracy: 0.9557
Epoch 9/10
78/78 [==============================] - 295s 4s/step - loss: 0.0967 - accuracy: 0.9678
Epoch 10/10
78/78 [==============================] - 301s 4s/step - loss: 0.0988 - accuracy: 0.9694


In [18]:
model.evaluate(valid_tensors,valid_targets)



20/20 [==============================] - 20s 1s/step - loss: 0.1733 - accuracy: 0.9582


[0.1732555478811264, 0.9581993818283081]

In [20]:
predictions = model.predict(test_tensors)
predictions

array([9.9349493e-01, 1.2467636e-04, 1.7871962e-04, 1.1037553e-04,
       8.2486006e-04, 9.7786215e-06, 8.4904234e-05, 1.8679566e-03,
       1.3943661e-04, 3.1643347e-03], dtype=float32)

In [ ]:
predictions